In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

import os

from PIL import Image

Using TensorFlow backend.


In [5]:
def load_data():
        data = []
        paths = []
        for r, d, f in os.walk(r"D:\Downloads\anime-faces\data"):
            for file in f:
                if '.png' in file:
                    paths.append(os.path.join(r, file))

        for path in paths:
            img = Image.open(path)
            x = np.array(img)
            x = x[...,:3]
            if(x.shape == (64,64,3)):
                data.append(x)

        x_train = np.array(data)
        x_train = x_train.reshape(len(data),64,64,3)
        return x_train

In [6]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 64
        self.img_cols = 64
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 50

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()
        generator = self.generator

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()
        model.add(Dense(64 * 4 * 4, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((4, 4, 64)))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(64, kernel_size=(3,3), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(3, kernel_size=(2,2), padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)
    

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        X_train = load_data()

        # Rescale -1 to 1
        X_train = X_train / 256

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 3, 3
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        gen_imgs = np.array(gen_imgs) * 256
        gen_imgs = gen_imgs.astype(int)

        fig=plt.figure(figsize=(20, 20))
        for i in range(1, c*r+1):
            img = gen_imgs[i-1]
            fig.add_subplot(r, c, i)
            plt.imshow(img)
        fig.savefig(r"C:\Users\Vee\Desktop\python\GAN\epoch_%d.png" % epoch)
        plt.close()
        self.generator.save("generator.h5")

In [7]:
dcgan = DCGAN()
dcgan.train(epochs=15001, batch_size=60, save_interval=100)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
batch_normalization_9 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 16, 16, 64)       

C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0

0 [D loss: 1.318786, acc.: 35.83%] [G loss: 0.418080]


C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.912841, acc.: 53.33%] [G loss: 0.948953]
2 [D loss: 0.546666, acc.: 69.17%] [G loss: 1.316187]
3 [D loss: 0.398449, acc.: 83.33%] [G loss: 1.499226]
4 [D loss: 0.323159, acc.: 89.17%] [G loss: 1.350314]
5 [D loss: 0.310970, acc.: 90.00%] [G loss: 0.966571]
6 [D loss: 0.258590, acc.: 92.50%] [G loss: 0.915175]
7 [D loss: 0.196794, acc.: 94.17%] [G loss: 1.053954]
8 [D loss: 0.214155, acc.: 95.83%] [G loss: 1.171052]
9 [D loss: 0.257280, acc.: 91.67%] [G loss: 1.297943]
10 [D loss: 0.298187, acc.: 87.50%] [G loss: 1.603608]
11 [D loss: 0.295474, acc.: 88.33%] [G loss: 1.379103]
12 [D loss: 0.503006, acc.: 77.50%] [G loss: 2.142339]
13 [D loss: 0.397123, acc.: 80.83%] [G loss: 2.385802]
14 [D loss: 0.191733, acc.: 97.50%] [G loss: 2.417748]
15 [D loss: 0.258827, acc.: 90.83%] [G loss: 2.260799]
16 [D loss: 0.190870, acc.: 94.17%] [G loss: 2.270734]
17 [D loss: 0.234469, acc.: 94.17%] [G loss: 2.632495]
18 [D loss: 0.173706, acc.: 93.33%] [G loss: 2.944223]
19 [D loss: 0.15597

...

14799 [D loss: 0.535655, acc.: 74.17%] [G loss: 2.788141]
14800 [D loss: 0.577826, acc.: 71.67%] [G loss: 3.371426]
14801 [D loss: 0.366329, acc.: 83.33%] [G loss: 3.157174]
14802 [D loss: 0.413024, acc.: 79.17%] [G loss: 3.432888]
14803 [D loss: 0.330667, acc.: 88.33%] [G loss: 2.374130]
14804 [D loss: 0.417319, acc.: 80.00%] [G loss: 3.097157]
14805 [D loss: 0.396670, acc.: 82.50%] [G loss: 3.018436]
14806 [D loss: 0.594594, acc.: 69.17%] [G loss: 2.919077]
14807 [D loss: 0.324030, acc.: 88.33%] [G loss: 3.065488]
14808 [D loss: 0.319355, acc.: 87.50%] [G loss: 2.811528]
14809 [D loss: 0.276613, acc.: 90.83%] [G loss: 2.411017]
14810 [D loss: 0.526089, acc.: 75.83%] [G loss: 2.987319]
14811 [D loss: 0.300602, acc.: 88.33%] [G loss: 3.544143]
14812 [D loss: 0.373722, acc.: 81.67%] [G loss: 2.698003]
14813 [D loss: 0.418574, acc.: 81.67%] [G loss: 2.972324]
14814 [D loss: 0.746052, acc.: 60.83%] [G loss: 2.711846]
14815 [D loss: 0.288200, acc.: 86.67%] [G loss: 3.210389]
14816 [D loss:

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


14900 [D loss: 0.333829, acc.: 82.50%] [G loss: 2.831148]
14901 [D loss: 0.460677, acc.: 77.50%] [G loss: 2.466211]
14902 [D loss: 0.388990, acc.: 80.83%] [G loss: 2.912876]
14903 [D loss: 0.457050, acc.: 76.67%] [G loss: 3.167829]
14904 [D loss: 0.465678, acc.: 79.17%] [G loss: 1.843960]
14905 [D loss: 0.314750, acc.: 90.00%] [G loss: 2.624287]
14906 [D loss: 0.433742, acc.: 83.33%] [G loss: 3.274860]
14907 [D loss: 0.350398, acc.: 84.17%] [G loss: 3.590038]
14908 [D loss: 0.541006, acc.: 71.67%] [G loss: 2.669178]
14909 [D loss: 0.472930, acc.: 75.83%] [G loss: 2.582424]
14910 [D loss: 0.384192, acc.: 82.50%] [G loss: 2.654237]
14911 [D loss: 0.264735, acc.: 87.50%] [G loss: 3.366058]
14912 [D loss: 0.382555, acc.: 84.17%] [G loss: 2.864513]
14913 [D loss: 0.319769, acc.: 85.83%] [G loss: 2.601708]
14914 [D loss: 0.644831, acc.: 74.17%] [G loss: 2.777364]
14915 [D loss: 0.365370, acc.: 82.50%] [G loss: 2.743350]
14916 [D loss: 0.422864, acc.: 82.50%] [G loss: 2.074727]
14917 [D loss:

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


15000 [D loss: 0.405249, acc.: 82.50%] [G loss: 2.106130]
